In [1]:
import keras
from keras import layers
import numpy as np

latent_dim = 100
height = 32
width = 32
channels = 3

adam = keras.optimizers.Adam(lr=0.0002, beta_1=0.5, clipvalue=1.0, decay=1e-8)

#初期化用
init = keras.initializers.RandomNormal(stddev=0.02)

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.


In [6]:
#生成ネットワーク
generator_input = keras.Input(shape=(latent_dim,))

#潜在変数を4x4 256-channels 特徴マップへ変換
x = layers.Dense(256 * 4 * 4, kernel_initializer=init, bias_initializer='zeros')(generator_input)
x = layers.LeakyReLU(0.2)(x)
x = layers.Reshape((4, 4, 256))(x)

#4x4 256-channelsを8x8 128-channelsの特徴マップへDeconvolutionで変換
x = layers.Conv2DTranspose(128, 5, strides=2, padding='same', kernel_initializer=init, bias_initializer='zeros')(x)
x = layers.LeakyReLU(0.2)(x)

#8x8 128-channelsを16x16 64-channelsの特徴マップへDeconvolutionで変換
x = layers.Conv2DTranspose(64, 5, strides=2, padding='same', kernel_initializer=init, bias_initializer='zeros')(x)
x = layers.LeakyReLU(0.2)(x)

#16x16 64-channelsを32x32 32-channelsの特徴マップへDeconvolutionで変換
x = layers.Conv2DTranspose(32, 5, strides=2, padding='same', kernel_initializer=init, bias_initializer='zeros')(x)
x = layers.LeakyReLU(0.2)(x)

#32x32 32-channelsを３２x32 3-channelsの特徴マップへ変換
x = layers.Conv2D(channels, 5, activation='tanh', padding='same', kernel_initializer=init, bias_initializer='zeros')(x)
generator = keras.models.Model(generator_input, x)
generator.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 100)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 4096)              413696    
_________________________________________________________________
leaky_re_lu_17 (LeakyReLU)   (None, 4096)              0         
_________________________________________________________________
reshape_5 (Reshape)          (None, 4, 4, 256)         0         
_________________________________________________________________
conv2d_transpose_13 (Conv2DT (None, 8, 8, 128)         819328    
_________________________________________________________________
leaky_re_lu_18 (LeakyReLU)   (None, 8, 8, 128)         0         
_________________________________________________________________
conv2d_transpose_14 (Conv2DT (None, 16, 16, 64)        204864    
__________

In [ ]:
#識別ネットワーク
discriminator_input = layers.Input(shape=(height, width, channels))
x = layers.Conv2D(64, 5, strides=2,padding='same',kernel_initializer=init)(discriminator_input)
x = layers.LeakyReLU(0.2)(x)
x = layers.Dropout(0.3)(x)
x = layers.Conv2D(128, 5, strides=2,padding='same')(x)
x = layers.LeakyReLU(0.2)(x)
x = layers.Dropout(0.3)(x)
x = layers.Conv2D(256, 5, strides=2,padding='same')(x)
x = layers.LeakyReLU(0.2)(x)
x = layers.Dropout(0.3)(x)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(1, activation='sigmoid')(x)

discriminator = keras.models.Model(discriminator_input, x)
discriminator.summary()


discriminator_optimizer  = adam
discriminator.compile(optimizer=discriminator_optimizer, loss='binary_crossentropy')